In [2]:
!pip install riotwatcher

In [3]:
from riotwatcher import LolWatcher, ApiError #
from pprint import pprint
from tqdm import tqdm #
import datetime
from contextlib import contextmanager


import pandas as pd #
import sys
import os
import csv
import pickle

import time

from collections import defaultdict

KST = datetime.timezone(datetime.timedelta(hours=9))

@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f"[{name}] done in {time.time() - t0:.3f} s")
    
data_path = "./data"

## 1. 소환사 정보 가져오기

In [4]:
os.listdir(data_path)

['DIAMOND_II_puuids.csv',
 'DIAMOND_III_puuids.csv',
 'MASTER_I_puuids.csv',
 'Untitled.ipynb',
 'master_puidList',
 'Summoner_Info_2021_11_24_23h_29m_16s.csv',
 '.ipynb_checkpoints',
 'DIAMOND_I_puuids.csv']

In [5]:
for fileName in os.listdir(data_path):
    if "Summoner_Info" in fileName:
        summoner_df = pd.read_csv(os.path.join(data_path, fileName))
        break

### 1-1. 티어별 유저 수 확인

In [6]:
summoner_df

,tier,queue,division,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,CHALLENGER,RANKED_SOLO_5x5,I,p090Y0IkJgORbrNzTBOeKPFpvBcMIc1CtiVu7NDiU_cv0w,JustLikeThatKR,976,806,746,True,False,False,False
1,CHALLENGER,RANKED_SOLO_5x5,I,L0iHC8DjfxeGKk2r3BoqU2A8GHtzbWLIh5l1rmVxs9vg6Y...,검색 구글 제드팀,950,153,92,False,False,False,False
2,CHALLENGER,RANKED_SOLO_5x5,I,oiTkjcprSh4guOT-0TgHmAGhD9EWMuG2uopx_a7552_8SF8,Gen G Quid,949,1643,1565,True,False,False,False
3,CHALLENGER,RANKED_SOLO_5x5,I,LVTnRLyW-FVQ3u-l5KVNsyIuILOGIdm078ANJdib0mZvNLo,bao1,1227,383,302,False,False,False,False
4,CHALLENGER,RANKED_SOLO_5x5,I,01Wuh8h-kD-JJrF7WeLGPm2zRXBSqianWyEAJXK2WjcMLl...,Gotchaoflegends,1048,221,160,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
4914004,IRON,RANKED_SOLO_5x5,IV,-i3b9hLq05LzfGHTZ0inY_IPBkwHo8Am-Jvo6H2FmBNoSf0,아이언3부계정,34,24,79,False,False,False,False
4914005,IRON,RANKED_SOLO_5x5,IV,x96VYGPARvgYeCUh0RoJFSZLwIjwU_KB71I4Fd2_fAHb6d0,이건그레이새기야,94,127,166,False,False,False,False
4914006,IRON,RANKED_SOLO_5x5,IV,biGTp84JOkB0wGkZ1cKBozFT0-62E2S_XQ11LdGXHfBlChc,화북동김카사디안,35,36,57,False,False,False,False
4914007,IRON,RANKED_SOLO_5x5,IV,yLA0oQFm0kN9OH5NK6bHTpJ88ezFbKOS9oKmiB-a-UVBgyk,vlp loo,29,4,12,False,False,False,False


In [7]:
# 티어별 유저수
summoner_df.groupby('tier').count()

,queue,division,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
tier,,,,,,,,,,,
BRONZE,799661,799661,799661,799660,799661,799661,799661,799661,799661,799661,799661
CHALLENGER,300,300,300,300,300,300,300,300,300,300,300
DIAMOND,70241,70241,70241,70241,70241,70241,70241,70241,70241,70241,70241
GOLD,1628560,1628560,1628560,1628560,1628560,1628560,1628560,1628560,1628560,1628560,1628560
GRANDMASTER,700,700,700,700,700,700,700,700,700,700,700
IRON,67869,67869,67869,67869,67869,67869,67869,67869,67869,67869,67869
MASTER,6049,6049,6049,6049,6049,6049,6049,6049,6049,6049,6049
PLATINUM,628141,628141,628141,628141,628141,628141,628141,628141,628141,628141,628141
SILVER,1712488,1712488,1712488,1712488,1712488,1712488,1712488,1712488,1712488,1712488,1712488


In [8]:
# 티어 구간별 유저수
tiers = ['BRONZE', 'CHALLENGER', 'DIAMOND', 'GOLD', 'GRANDMASTER', 'IRON', 'MASTER', 'PLATINUM', 'SILVER']
for i, dfByTier in enumerate([summoner_df[summoner_df['tier']==tier] for tier in tiers]):
    print('='*100)
    print(tiers[i])
    display(dfByTier.groupby('division').count())
    print('='*100)

BRONZE


,tier,queue,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
division,,,,,,,,,,,
I,243996,243996,243996,243996,243996,243996,243996,243996,243996,243996,243996
II,241079,241079,241079,241079,241079,241079,241079,241079,241079,241079,241079
III,171347,171347,171347,171347,171347,171347,171347,171347,171347,171347,171347
IV,143239,143239,143239,143238,143239,143239,143239,143239,143239,143239,143239


CHALLENGER


,tier,queue,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
division,,,,,,,,,,,
I,300,300,300,300,300,300,300,300,300,300,300


DIAMOND


...

,tier,queue,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
division,,,,,,,,,,,
I,6049,6049,6049,6049,6049,6049,6049,6049,6049,6049,6049


PLATINUM


,tier,queue,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
division,,,,,,,,,,,
I,72086,72086,72086,72086,72086,72086,72086,72086,72086,72086,72086
II,70963,70963,70963,70963,70963,70963,70963,70963,70963,70963,70963
III,121379,121379,121379,121379,121379,121379,121379,121379,121379,121379,121379
IV,363713,363713,363713,363713,363713,363713,363713,363713,363713,363713,363713


SILVER


,tier,queue,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
division,,,,,,,,,,,
I,312721,312721,312721,312721,312721,312721,312721,312721,312721,312721,312721
II,421364,421364,421364,421364,421364,421364,421364,421364,421364,421364,421364
III,405467,405467,405467,405467,405467,405467,405467,405467,405467,405467,405467
IV,572936,572936,572936,572936,572936,572936,572936,572936,572936,572936,572936


In [9]:
# 중복 확인
len(summoner_df['summonerId']), len(set(summoner_df['summonerId']))

(4914009, 4914009)

## 2. 수집 정보 설정

In [14]:

riot_token = "RGAPI-2a6b99be-6e31-41ee-8f37-cdd2389ae608" # 토큰 넣어주세요. 수집 데이터가 많을 때는 재발급하고 넣어주세요

lol_watcher = LolWatcher(riot_token)
my_region = 'kr'

me = lol_watcher.summoner.by_name(my_region, 'hide on bush')
pprint(me)


{'accountId': 'i8-NWVUQrY8UdYFxEUo2e63TDHdtDB41zHB0ChLhM-hP',
 'id': 'GbesqslvywKBlcXQYhCvVnwN_tTJ3iL79zKeMawG15An9g',
 'name': 'Hide on bush',
 'profileIconId': 6,
 'puuid': 'AUZp_JY-Akey--Qg-b0B4cmcv75ZcUkL_yDrmebJuLmXhdNZRFhbf9KbZDckAPOpDAgo2K9fcJ4zrA',
 'revisionDate': 1638206759137,
 'summonerLevel': 508}


In [15]:
queue = "RANKED_SOLO_5x5"
queueId = 420 # "5v5 Ranked Solo games" ID -> https://static.developer.riotgames.com/docs/lol/queues.json

#tiers = ['DIAMOND', 'PLATINUM', 'GOLD', 'SILVER', 'BRONZE', 'IRON']
#top_tier = ["CHALLENGER", "GRANDMASTER", "MASTER"]
#divisions = ['I', 'II', 'III', 'IV']

targetTier = 'MASTER' # 티어 넣어주세요
targetDivision = 'I' # 구간 넣어주세요

In [16]:
def collectPuuid(summoner_df, targetTier, targetDivision):
    print(f'Collect Puuid')
    print(f'    Target tier : {targetTier} {targetDivision}')
    summonerNameList = summoner_df[(summoner_df.tier==targetTier) & (summoner_df.division==targetDivision)]['summonerName']
    puuids = []
    
    with timer(f'{targetTier} {targetDivision} : puuid'):
        for summonerName in tqdm(summonerNameList, file=sys.stdout):
            try:
                puuid = lol_watcher.summoner.by_name(my_region, summonerName)['puuid']
            except:
                continue
            puuids.append(puuid)
        
    print()
    return puuids

def collectMatchData(matchDataDict, puuidList, targetTier, targetDivision, continueArgs=None):
    if continueArgs is None: assert len(matchDataDict) == 0, "수집 완료된 Dict 입니다."
    print(f'Collect Match data')
    print(f'    Target tier : {targetTier} {targetDivision}')

    now = datetime.datetime.now(KST)
    
    with timer(f'{targetTier} {targetDivision} : Match Data'):
        
        if continueArgs is not None:
            puuids = puuidList[continueArgs[0]:]
            matchIdSet = continueArgs[1]
        else :
            puuids = puuidList[:]
            matchIdSet = set() # 중복 수집 방지
        
        data_cnt = sum(len(data) for data in matchDataDict.values())
        pbar = tqdm(puuids, file=sys.stdout, total=len(puuids))
        for puuid in pbar:
            
            if continueArgs is not None: 
                startIndex = continueArgs[2]
            else : 
                startIndex = 0
                
            count = 100

            while True:
                try: 
                    matchIdList = lol_watcher.match.matchlist_by_puuid('asia', puuid, start=startIndex, count=count, queue=queueId) # 매치 아이디 
                except ApiError as err: 
                    return matchDataDict, now, (puuidList.index(puuid), matchIdSet,  startIndex, err.response)

                if len(matchIdList) == 0: break

                for i, matchId in enumerate(matchIdList):
                    if matchId in matchIdSet: continue # 중복 수집 방지
                    matchIdSet.add(matchId)
                    
                    try:
                        matchData = lol_watcher.match.by_id('asia', matchId) # 매치 데이터
                    except ApiError as err:
                        return matchDataDict, now, (puuidList.index(puuid), matchIdSet, startIndex+i, err.response)
                    
                    version = matchData['info']['gameVersion']
                    matchDataDict[version].append(matchData)
                    
                    data_cnt += 1
                    pbar.set_description('    ' + f'current MatchData -> {data_cnt}개')
                    
                startIndex += count
                
            # return matchDataDict, now, None # 한명만 수집할 때
        
    print()
    return matchDataDict, now, None

## 3. 데이터 수집

In [201]:
puuidList = collectPuuid(summoner_df, targetTier, targetDivision)

Collect Puuid
    Target tier : MASTER I
100%|██████████| 6049/6049 [2:00:30<00:00,  1.20s/it]   
[MASTER I : puuid] done in 7230.517 s



In [208]:
len(puuidList)

5654

In [203]:
# puuidList csv로 저장하기

puuids = ["puuids"]
with open(os.path.join(data_path, f"{targetTier}_{targetDivision}_puuids.csv"), "w") as f:
    writer = csv.writer(f, delimiter="\n")
    writer.writerow(puuids)
    writer.writerow(puuidList)

In [17]:
# csv 로드해서 puuidList 만들기

puuidList_data = []
with open(os.path.join(data_path, f"{targetTier}_{targetDivision}_puuids.csv"), "r") as f:
    reader = csv.reader(f)
    for row in reader:
        puuidList_data.append(row[0])
puuidList = puuidList_data[1:]

In [205]:
# master puuid pickle 로드하기

import pickle

with open(os.path.join(data_path, 'master_puidList'), 'rb') as f:
    puuidList = pickle.load(f)
    print('load list')

load list


In [18]:
puuidList = puuidList[:500]

In [19]:
puuidList[0]

'JaEtuZY_GiimRE5rhR6FyN3Cw3il-2qYOWtpb0lhiDpPRt2yjR94MBbtahWsTEbrGvweOUKtCOcmWQ'

In [20]:
len(puuidList)

500

In [21]:
# 처음부터 다시 수집하고 싶을 때 이 셀을 실행해 초기화해주세요
matchDataDict = defaultdict(list)
continueArgs = None

In [ ]:
# 에러가 발생해 중간에 멈추면 이 셀을 다시 실행해주세요 (토큰 문제시 재발급)
# 에러 코드 설명 : https://developer.riotgames.com/docs/portal

matchDataDict, now, continueArgs = collectMatchData(matchDataDict, puuidList, targetTier, targetDivision, continueArgs)
if continueArgs is not None:
    print()
    pprint(continueArgs[-1].json())
    print('\033[31m' + "Error. Please restart" + '\033[0m')

Collect Match data
    Target tier : MASTER I
    current MatchData -> 24212개:   8%|██████████▊                                                                                                                                   | 33/431 [42:20<7:44:50, 70.08s/it]]

In [24]:
continueArgs

(69,
 {'KR_5518077554',
  'KR_5483595785',
  'KR_5317664511',
  'KR_5421200370',
  'KR_5322698219',
  'KR_5495717424',
  'KR_5443436161',
  'KR_5419009388',
  'KR_5483162826',
  'KR_5545010623',
  'KR_5484145743',
  'KR_5513161563',
  'KR_5531023489',
  'KR_5406357139',
  'KR_5464788406',
  'KR_5412860270',
  'KR_5344110053',
  'KR_5527076091',
  'KR_5491250534',
  'KR_5460450898',
  'KR_5495190317',
  'KR_5521919458',
  'KR_5411034494',
  'KR_5546761555',
  'KR_5312834209',
  'KR_5544974407',
  'KR_5306681281',
  'KR_5387592900',
  'KR_5404304528',
  'KR_5473498530',
  'KR_5379276174',
  'KR_5390567906',
  'KR_5511301473',
  'KR_5446017966',
  'KR_5329654016',
  'KR_5507260450',
  'KR_5379073778',
  'KR_5373995818',
  'KR_5515235461',
  'KR_5408894495',
  'KR_5542124596',
  'KR_5358996996',
  'KR_5505723554',
  'KR_5502241966',
  'KR_5397850148',
  'KR_5497531459',
  'KR_5430157683',
  'KR_5437168980',
  'KR_5323877888',
  'KR_5489169705',
  'KR_5283387298',
  'KR_5317747000',
  'KR_5

In [25]:
matchDataDict.keys()

dict_keys(['11.23.409.111', '11.22.406.3587', '11.21.403.3002', '11.20.400.7328', '11.19.398.9466', '11.18.395.7538', '11.17.394.4489', '11.17.393.607', '11.16.390.1945', '11.15.389.2308', '11.15.388.2387', '11.14.385.9967', '11.14.384.6677', '11.13.382.1241', '11.19.398.2521', '11.15.387.5736'])

## 4. 저장

In [26]:
import json

file_name = f"{targetTier}_{targetDivision}_MatchData_{now.year}_{now.month}_{now.day}_{now.hour}h_{now.minute}m_{now.second}s.json"

with open(os.path.join(data_path, file_name), 'w') as fp:
    json.dump(matchDataDict, fp)